In [1]:
import sys
sys.path.insert(0, '../Tools/')

In [2]:
import data_prep
import model_tools

Using TensorFlow backend.
/anaconda2/envs/envPython3TF1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda2/envs/envPython3TF1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda2/envs/envPython3TF1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda2/envs/envPython3TF1/lib/p

In [3]:
import keras
from keras import backend as K
GPU_NO="0"
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.visible_device_list = GPU_NO
set_session(tf.Session(config=config))

In [4]:
mnist=data_prep.mnist_data_prep()
letters=data_prep.letters_prep()

In init of mnist_data_prep
In init of letters_prep


In [5]:
import numpy as np
import plotly.plotly as py
from plotly.graph_objs import *
import plotly
from plotly.graph_objs import Scatter, Layout
import os
from IPython.display import display, HTML

ModuleNotFoundError: No module named 'plotly'

In [ ]:
def plot_3D(features,meta_data,output_file,alpha=False):
    meta_data=np.array(meta_data)
    colors=data_prep.colors*256
    x=features[:,0]
    y=features[:,1]
    z=np.zeros(len(features[:,1]))
    
    feature_trace=[]
    feature_anno=[]
    plane_trace=[]

    for i in list(set(meta_data)):
        feature_trace.append(dict(
                                      x = x[meta_data==i], 
                                      y = y[meta_data==i],
                                      z = z[meta_data==i],
                                      marker = dict(
                                                        color = "rgb("+str(colors[i][0])+","+str(colors[i][1])+","+str(colors[i][2])+")", 
                                                        size = 1,
                                                        showscale = False,
                                                    ), 
                                      mode = "markers",
                                      name = str(i),
                                      type = "scatter3d"
                                ))
        feature_anno.append(dict(
                                    x=i*0.1,
                                    y=-0.1,
                                    text=str(i),
                                    showarrow=False,
                                    font=dict(
                                        family='Courier New, monospace',
                                        size=16,
                                        color="rgb("+str(colors[i][0])+","+str(colors[i][1])+","+str(colors[i][2])+")"
                                )))
        min_x,max_x=np.min(x[meta_data<10]),np.max(x[meta_data<10])
        min_y,max_y=np.min(y[meta_data<10]),np.max(y[meta_data<10])
        all_x=np.linspace(min_x*1.5, max_x*1.5,5)
        all_y=np.linspace(min_y*1.5, max_y*1.5,5)
        
        e_=[]
        for y_ in all_y.tolist():
            e_.append(np.dot(np.transpose(np.array([all_x,np.ones(len(all_x))*y_])),pred_weights[:,i]))
            
        plane_trace.append(dict(
            x = all_x.tolist(),
            y = all_y.tolist(),
            z = e_,
            colorscale = [
                            [0, "rgba("+str(colors[i][0])+","+str(colors[i][1])+","+str(colors[i][2])+",0.1)"], 
                            [1, "rgba("+str(colors[i][0])+","+str(colors[i][1])+","+str(colors[i][2])+",0.1)"], 
                        ],
            showscale = False,
            name = str(i)+" Plane", 
            type = "surface"
        ))
        
    if alpha:
        e_=(np.ones((len(all_y),len(all_x)))*100).tolist()

        plane_trace.append(dict(
                            x = all_x.tolist(), 
                            y = all_y.tolist(),
                            z = e_,
                            colorscale = [[0, "rgb(0,0,0)"],[1, "rgb(0,0,0)"]],
                            showscale = False,
                            name = "Alpha Plane",
                            type = "surface",
                        ))

    l=dict(
            autorange=True,
            showgrid=False,
            zeroline=False,
            showline=False,
            ticks='',
            showticklabels=False,
            title = ''
        )

    layout = plotly.graph_objs.Layout(
        scene=dict(
                    xaxis=l,
                    yaxis=l,
                    zaxis=l
                ),
        showlegend=False,
        annotations=feature_anno,
        updatemenus=[
                    {'type': 'buttons',
                     'buttons': [{'label': 'Show Planes',
                                   'method': 'animate',
                                   'args': [None,{'frame':{'duration':1400, 'redraw':True}}]}]}]
    )


    plotly.offline.plot({
                          'data': Data(plane_trace),
                            "layout":layout,
                            'frames':[{'data': Data(feature_trace)},
                                      {'data': Data(plane_trace)},
                                      {'data': Data(feature_trace)},
                                     ]
                        })
    
    os.rename('temp-plot.html',output_file)
    display(HTML("<iframe width=\"100%\" height=\"500\" src="+output_file+"></iframe>"))

In [6]:
random_model_no='0'
dataset_type='Devanagari'

In [7]:
vanilla_lenet_pp=keras.models.load_model('LeNet++/Models/Vanilla_'+random_model_no+'.h5py')
mnist_fc_output=model_tools.extract_features(vanilla_lenet_pp,mnist.X_train,layer_name=['fc'])
pred_weights=vanilla_lenet_pp.get_layer('pred').get_weights()[0]

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [8]:
plot_3D(mnist_fc_output,mnist.labels_train,output_file='No_alpha_Plane.html',alpha=False)

NameError: name 'plot_3D' is not defined

In [10]:
plot_3D(mnist_fc_output,mnist.labels_train,output_file='With_alpha_Plane.html',alpha=True)

### Background Class

In [11]:
bg_lenet_pp=keras.models.load_model('LeNet++/Models/BG_'+random_model_no+'.h5py')
mnist_fc_output=model_tools.extract_features(bg_lenet_pp,mnist.X_train,layer_name=['fc'])
pred_weights=bg_lenet_pp.get_layer('pred').get_weights()[0]

In [12]:
letters_fc_output=model_tools.extract_features(bg_lenet_pp,letters.X_train,layer_name=['fc'])
labels=mnist.labels_train.tolist()
labels.extend((np.ones(len(letters.X_train))*10).astype(np.int32).tolist())

In [13]:
plot_3D(np.array(mnist_fc_output.tolist()+letters_fc_output.tolist()),labels,output_file='BG_Plane.html',alpha=False)